In [ ]:
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from math import comb
import pickle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import shap
%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['text.usetex'] = False

# Read the dataset

Before reading the dataset, dowload it using:
```
./dowload_data.sh
```

In [ ]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship",
             "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "class"]
df = pd.read_csv("adult.csv", sep=",", header=None, names=col_names, index_col=False)

In [ ]:
train = df.head(int(len(df) * 0.7))
test = df.tail(len(df) - len(train))
resp_var = "class"
X_train = train.drop(resp_var, axis=1)
y_train = train[resp_var]
X_test = test.drop(resp_var, axis=1)
y_test = test[resp_var]

In [ ]:
final_cols = []
categorical_feats = ["workclass", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
to_drop = ["education"]
transformers = []
for column in X_train.columns:
    name = column
    trans = "passthrough"
    if column in categorical_feats:
        trans = OneHotEncoder()
        name = f"{column}_class"
    elif column in to_drop:
        trans = "drop"

    transformers.append((name, trans, [f"{column}"]))

    if trans != "drop":
        final_cols.append(column)
ct = ColumnTransformer(transformers, remainder="passthrough")
ct.fit(X_train)
# Encoder for the labels
le = LabelEncoder()
le.fit(y_train)

In [ ]:
X_train_trans = ct.transform(X_train)
X_test_trans = ct.transform(X_test)
y_train_trans = le.transform(y_train)
y_test_trans = le.transform(y_test)

# Feature selection

## Feature selection

To compute the results run:
```
python forest_train.py
python feat_selection.py
```

In [ ]:
range_n_splines = range(1, 11)
range_n_inter = range(0, 9)

In [ ]:
acc = np.load("precomputed/feat_selection.npy")

In [ ]:
dimension = (len(range_n_splines), len(range_n_inter))
mask = np.zeros(dimension)
for i, n_splines in enumerate(tqdm(range_n_splines)):
    for j, n_inter in enumerate(range_n_inter):
        if n_inter > comb(n_splines, 2):
            mask[i, j] = True
            continue

In [ ]:
accuracy_df = pd.DataFrame(acc, columns=range_n_inter, index=range_n_splines)
ax = sns.heatmap(accuracy_df, annot=True, mask=mask, cmap=sns.color_palette("Blues", as_cmap=True),
                 cbar_kws={'label': 'accuarcy'})
ax.set_xlabel("Number of interaction terms used")
ax.set_ylabel("Number of splines used")

# Sampling strategy

To replicate the experiments run:
```
python forest_train.py
python sampling_analysis.py
```

## Setup

In [ ]:
sampling_methods = ["all", "quantile", "equal", "kmeans", "equi_size"]
range_m = range(50, 5001, 250)

In [ ]:
with open('precomputed/sampling_comparison.pickle', 'rb') as f:
    acc_methods = pickle.load(f)

## Plot

In [ ]:
labels = [r"\emph{All-Thresholds}", r"\emph{Quantile}", r"\emph{Equi-Width}", r"\emph{$k$-Means}", "\emph{Equi-Size}"]
colors = sns.color_palette(n_colors=len(sampling_methods))
for i, sampling_method in enumerate(sampling_methods):
    plt.plot(range_m, acc_methods[sampling_method], 'o--', color=colors[i], label=labels[i])
plt.xlabel("$K$")
plt.ylabel("Accuracy")
plt.legend()

# Global explanation

## GEF

To replicate the results run:
```
python forest_train.py
python final_explainer.py
```

In [ ]:
with open("precomputed/explainer.pickle", "rb") as f:
    explainer = pickle.load(f)

In [ ]:
final_cols = ct.get_feature_names_out().copy()
final_cols[14] = "MS-Married"
final_cols[47] = "CapitalGain"
final_cols[11] = "EducationNum"
final_cols[0] = "Age"

In [ ]:
sample_index = 0
sample = X_train_trans[sample_index].reshape(1, -1)

In [ ]:
n_row, n_col = 2, 2

fig = plt.figure(figsize=(13, 8), tight_layout=False)

lines = []

terms = [(i, x) for i, x in enumerate(explainer.gam.terms) if not x.isintercept and not x.istensor]
terms.sort(key=lambda x: x[1].feature)
c1, c2, c3 = sns.color_palette(n_colors=3)

plot_index = 0
axes = []
for i, term in enumerate(explainer.gam.terms):
    if plot_index == 4:
        break
    if term.isintercept or term.istensor:
        continue

    ax = fig.add_subplot(n_row, n_col, plot_index + 1, sharey = axes[-1] if plot_index % n_col != 0 else None)
    plt.setp(ax.get_yticklabels(), visible=plot_index % n_col == 0)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    print(term.feature)

    # Spline print
    grid = explainer.gam.generate_X_grid(term=i, meshgrid=term.istensor)
    pdep, confi = explainer.gam.partial_dependence(term=i, X=grid, width=0.95, meshgrid=term.istensor)

    conf_u = ax.plot(grid[:, term.feature], confi[:, 0], ls="--", c=c2, zorder=1)
    conf_l = ax.plot(grid[:, term.feature], confi[:, 1], label="95% width confidence interval", ls="--", c=c2, zorder=1)
    l1 = ax.plot(grid[:, term.feature], pdep, label="Spline learned", lw=2, c=c1, zorder=2)
    ax.set_title(final_cols[term.feature])

    # Print the sample
    """
    x_point = sample[0, term.feature]  # col vector
    y_point = explainer.gam.partial_dependence(term=i, X=sample)

    plt.vlines(x_point, ax.get_ylim()[0], y_point, linestyle="dashed", color=c3)
    plt.hlines(y_point, ax.get_xlim()[0], x_point, linestyle="dashed", color=c3)
    ax.scatter(x_point, y_point, label="Sample under investigation", color=c3, zorder=3)
    """

    plot_index += 1
    axes.append(ax)

plt.subplots_adjust(hspace=0.3)
file_out = "plots/generators.pdf"
params = {'legend.fontsize': 18,
          'figure.figsize': (20, 5),
          'axes.titlesize': 18,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20}
plt.rcParams.update(params)
plt.legend(loc='upper center', bbox_to_anchor=(-0.35, 2.7), ncol=3)
plt.savefig(file_out)

## SHAP

To replicate the results run:
```
python forest_train.py
python compute_shap.py
```

### Setup

In [ ]:
with open('precomputed/shap_values_training.pickle', 'rb') as f:
    shap_values = pickle.load(f)
with open('precomputed/shap_explainer_training.pickle', 'rb') as f:
    shap_explainer = pickle.load(f)

### Plot

In [ ]:
# visualize the first prediction's explanation
shap.plots.force(shap_explainer.expected_value[1], shap_values.values[0, :, 1], matplotlib=True)

In [ ]:
shap_values = shap_values[:, :, 1]

In [ ]:
n_row, n_col = 2, 2

fig = plt.figure(figsize=(13, 8))

lines = []

terms = [(i, x) for i, x in enumerate(explainer.gam.terms) if not x.isintercept and not x.istensor]
terms.sort(key=lambda x: x[1].feature)
c1, c2, c3 = sns.color_palette(n_colors=3)

plot_index = 0
axes = []
for i, term in enumerate(explainer.gam.terms):
    if plot_index == 4:
        break
    if term.isintercept or term.istensor:
        continue

    ax = fig.add_subplot(n_row, n_col, plot_index + 1, sharey = axes[-1] if plot_index % n_col != 0 else None)

    # Shap scatter print
    shap.plots.scatter(shap_values[:, term.feature], ax=ax, show=False, hist=False, color=c1)
    shap_plot = ax
    
    plt.setp(ax.get_yticklabels(), visible=plot_index % n_col == 0)
    ax.tick_params(labelsize=20)

    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.set_title(final_cols[term.feature])

    # Print the sample
    """
    x_point = shap_values[sample_index, term.feature].data
    y_point = shap_values[sample_index, term.feature].values

    plt.vlines(x_point, ax.get_ylim()[0], y_point, linestyle="dashed", color=c2)
    plt.hlines(y_point, ax.get_xlim()[0], x_point, linestyle="dashed", color=c2)
    sample_plot = ax.scatter(x_point, y_point, label="Sample under investigation", color=c2, zorder=3)
    """

    plot_index += 1
    axes.append(ax)
    
params = {'legend.fontsize': 18,
          'figure.figsize': (20, 5),
          'axes.titlesize': 18}

plt.rcParams.update(params)
plt.subplots_adjust(hspace=0.3)
file_out = "plots/shap.pdf"
dummy_shap_plot = Line2D([0], [0], marker='o', color=c1, label='SHAP values', lw=0)
plt.legend(handles=[dummy_shap_plot], loc='upper center', bbox_to_anchor=(-1.0, 2.7), ncol=3, fontsize=14)
plt.savefig(file_out)